In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.batch1 = nn.BatchNorm2d(num_features = 8)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)
        self.batch2 = nn.BatchNorm2d(num_features = 8)
        self.drop1 = nn.Dropout2d(0.2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 20, 3, padding=1)
        self.batch3 = nn.BatchNorm2d(num_features = 20)
        self.conv4 = nn.Conv2d(20, 20, 3, padding=1)
        self.batch4 = nn.BatchNorm2d(num_features = 20)
        self.drop2 = nn.Dropout2d(0.2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(20, 20, 3)
        self.batch5 = nn.BatchNorm2d(num_features = 20)
        self.conv6 = nn.Conv2d(20, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1(x)))
        x = self.batch2(F.relu(self.conv2(x)))
        x = self.drop1(self.pool1(x))
        x = self.batch3(F.relu(self.conv3(x)))
        x = self.batch4(F.relu(self.conv4(x)))
        x = self.drop2(self.pool2(x))
        x = self.batch5(F.relu(self.conv5(x)))
        x = F.relu(self.conv6(x))
        x = self.conv7(x)
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [77]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             584
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
         Dropout2d-6            [-1, 8, 14, 14]               0
            Conv2d-7           [-1, 20, 14, 14]           1,460
       BatchNorm2d-8           [-1, 20, 14, 14]              40
            Conv2d-9           [-1, 20, 14, 14]           3,620
      BatchNorm2d-10           [-1, 20, 14, 14]              40
        MaxPool2d-11             [-1, 20, 7, 7]               0
        Dropout2d-12             [-1, 20, 7, 7]               0
           Conv2d-13             [-1, 20, 5, 5]           3,620
      BatchNorm2d-14             [-1, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
# I am turning OFF TQDM. I don't want progress bar for this project
# Please note that it will not affect your train/test accuracy
#from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [80]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0575, Accuracy: 9810/10000 (98.10%)


Test set: Average loss: 0.0420, Accuracy: 9854/10000 (98.54%)


Test set: Average loss: 0.0317, Accuracy: 9899/10000 (98.99%)


Test set: Average loss: 0.0295, Accuracy: 9893/10000 (98.93%)


Test set: Average loss: 0.0290, Accuracy: 9901/10000 (99.01%)


Test set: Average loss: 0.0230, Accuracy: 9918/10000 (99.18%)


Test set: Average loss: 0.0233, Accuracy: 9927/10000 (99.27%)


Test set: Average loss: 0.0226, Accuracy: 9927/10000 (99.27%)


Test set: Average loss: 0.0221, Accuracy: 9926/10000 (99.26%)


Test set: Average loss: 0.0232, Accuracy: 9923/10000 (99.23%)


Test set: Average loss: 0.0244, Accuracy: 9925/10000 (99.25%)


Test set: Average loss: 0.0214, Accuracy: 9932/10000 (99.32%)


Test set: Average loss: 0.0204, Accuracy: 9939/10000 (99.39%)


Test set: Average loss: 0.0209, Accuracy: 9935/10000 (99.35%)


Test set: Average loss: 0.0231, Accuracy: 9935/10000 (99.35%)


Test set: Average loss: 0.0204, Accurac